In [1]:
import pandas as pd 
import numpy as np 

In [2]:
reg = pd.read_csv('./data/md_ev_reg_zip.csv')
stations = pd.read_csv('./data/md_ev_stations.csv')

In [3]:
stations = stations[['Program','Station Name','ZipCode','Access']]
stations.head()

,Program,Station Name,ZipCode,Access
0,Electric Vehicle Charging Stations,Antwerpen Nissan - Clarksville,21029,Public
1,Electric Vehicle Supply Equipment Tax Credit P...,Capitol Fisker,20770,Public
2,Electric Vehicle Charging Stations,Hamilton Nissan,21740,Public
3,Electric Vehicle Charging Stations,Walgreens,21213,Public
4,Electric Vehicle Charging Stations,The Frederick Motor Company,21702,Public


In [4]:
reg.head()

,Year_Month,Fuel_Category,Zip_Code,Count
0,2020/07,Electric,19973,1
1,2020/07,Electric,20601,21
2,2020/07,Electric,20602,26
3,2020/07,Electric,20603,54
4,2020/07,Electric,20607,35


In [5]:
reg.loc[reg['Zip_Code'].isin(['UNKNOWN', 'VA', 'NY','RESTO','2001+']), 'Zip_Code'] = '0'
reg['Zip_Code'] = reg['Zip_Code'].fillna(0)
reg['Zip_Code'] = reg['Zip_Code'].astype(int)

In [6]:
reg

,Year_Month,Fuel_Category,Zip_Code,Count
0,2020/07,Electric,19973,1
1,2020/07,Electric,20601,21
2,2020/07,Electric,20602,26
3,2020/07,Electric,20603,54
4,2020/07,Electric,20607,35
...,...,...,...,...
80710,2025/03,Plug-In Hybrid,95628,1
80711,2025/03,Plug-In Hybrid,98112,1
80712,2025/03,Plug-In Hybrid,98370,1
80713,2025/03,Plug-In Hybrid,98626,1


In [7]:
merged = pd.merge(reg, stations, how='left', left_on='Zip_Code', right_on='ZipCode')
merged.head()

,Year_Month,Fuel_Category,Zip_Code,Count,Program,Station Name,ZipCode,Access
0,2020/07,Electric,19973,1,NaN,NaN,NaN,NaN
1,2020/07,Electric,20601,21,Electric Vehicle Supply Equipment Tax Credit P...,Sheehy Auto Store,20601.0,Public
2,2020/07,Electric,20601,21,Electric Vehicle Charging Stations,Waldorf Ford,20601.0,Public
3,2020/07,Electric,20601,21,Electric Vehicle Infrastructure Program (EVIP),Charles County Hilton,20601.0,Public
4,2020/07,Electric,20602,26,Electric Vehicle Charging Stations,Charles County Public Library - PD Brown Memor...,20602.0,Public


In [8]:
merged['Year_Month'] = pd.to_datetime(merged['Year_Month'], format='%Y/%m')

In [9]:
merged['year'] = merged['Year_Month'].dt.year
merged['month'] = merged['Year_Month'].dt.month

In [10]:
merged['Fuel_Category'] = merged['Fuel_Category'].apply(lambda x: 'BEV' if x=='Electric' else 'PHEV')

In [11]:
merged.dropna(axis=0)

,Year_Month,Fuel_Category,Zip_Code,Count,Program,Station Name,ZipCode,Access,year,month
1,2020-07-01,BEV,20601,21,Electric Vehicle Supply Equipment Tax Credit P...,Sheehy Auto Store,20601.0,Public,2020,7
2,2020-07-01,BEV,20601,21,Electric Vehicle Charging Stations,Waldorf Ford,20601.0,Public,2020,7
3,2020-07-01,BEV,20601,21,Electric Vehicle Infrastructure Program (EVIP),Charles County Hilton,20601.0,Public,2020,7
4,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Charles County Public Library - PD Brown Memor...,20602.0,Public,2020,7
5,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Walgreens,20602.0,Public,2020,7
...,...,...,...,...,...,...,...,...,...,...
98715,2025-03-01,PHEV,21771,220,Electric Vehicle Charging Stations,Century Ford,21771.0,Public,2025,3
98725,2025-03-01,PHEV,21784,233,Electric Vehicle Charging Stations,Walgreens,21784.0,Public,2025,3
98735,2025-03-01,PHEV,21801,126,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21801.0,Public,2025,3
98736,2025-03-01,PHEV,21804,76,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21804.0,Public,2025,3


In [12]:
simplified = merged.groupby(['year', 'month', 'Zip_Code', 'Fuel_Category']).agg({'Count': 'first'}).reset_index()
simplified

,year,month,Zip_Code,Fuel_Category,Count
0,2020,7,0,BEV,28
1,2020,7,0,PHEV,18
2,2020,7,19973,BEV,1
3,2020,7,19973,PHEV,1
4,2020,7,20601,BEV,21
...,...,...,...,...,...
80700,2025,3,98626,PHEV,1
80701,2025,3,99504,BEV,1
80702,2025,3,99505,BEV,1
80703,2025,3,99577,PHEV,1


In [13]:
monthly = merged.groupby(['Year_Month','Fuel_Category']).agg({'Count': 'sum'}).reset_index()
monthly

,Year_Month,Fuel_Category,Count
0,2020-07-01,BEV,36962
1,2020-07-01,PHEV,24286
2,2020-08-01,BEV,37895
3,2020-08-01,PHEV,24694
4,2020-09-01,BEV,38546
...,...,...,...
109,2025-01-01,PHEV,84759
110,2025-02-01,BEV,213450
111,2025-02-01,PHEV,85559
112,2025-03-01,BEV,217665


In [14]:
import plotly.express as px

In [15]:
px.line(monthly, x='Year_Month', y='Count', color='Fuel_Category', title='Monthly EV Registrations in Maryland by Fuel Type').show()

Now we cross-reference with [this database](https://simplemaps.com/data/us-zips.)

In [16]:
md_only = merged.dropna(axis=0)
md_only

,Year_Month,Fuel_Category,Zip_Code,Count,Program,Station Name,ZipCode,Access,year,month
1,2020-07-01,BEV,20601,21,Electric Vehicle Supply Equipment Tax Credit P...,Sheehy Auto Store,20601.0,Public,2020,7
2,2020-07-01,BEV,20601,21,Electric Vehicle Charging Stations,Waldorf Ford,20601.0,Public,2020,7
3,2020-07-01,BEV,20601,21,Electric Vehicle Infrastructure Program (EVIP),Charles County Hilton,20601.0,Public,2020,7
4,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Charles County Public Library - PD Brown Memor...,20602.0,Public,2020,7
5,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Walgreens,20602.0,Public,2020,7
...,...,...,...,...,...,...,...,...,...,...
98715,2025-03-01,PHEV,21771,220,Electric Vehicle Charging Stations,Century Ford,21771.0,Public,2025,3
98725,2025-03-01,PHEV,21784,233,Electric Vehicle Charging Stations,Walgreens,21784.0,Public,2025,3
98735,2025-03-01,PHEV,21801,126,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21801.0,Public,2025,3
98736,2025-03-01,PHEV,21804,76,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21804.0,Public,2025,3


In [17]:
zips = pd.read_csv('./data/uszips.csv')
zips

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33778,99923,55.98043,-130.03803,Hyder,AK,Alaska,True,NaN,25.0,0.6,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33779,99925,55.55398,-132.96276,Klawock,AK,Alaska,True,NaN,854.0,6.1,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33780,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1385.0,4.0,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Metlakatla
33781,99927,56.33305,-133.60044,Point Baker,AK,Alaska,True,NaN,18.0,1.5,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka


In [18]:
zips = zips[['zip', 'lat', 'lng', 'city', 'state_id','population','density']]


In [19]:
with_zip_stats = pd.merge(md_only, zips, how='left', left_on='Zip_Code', right_on='zip')

In [20]:
with_zip_stats

,Year_Month,Fuel_Category,Zip_Code,Count,Program,Station Name,ZipCode,Access,year,month,zip,lat,lng,city,state_id,population,density
0,2020-07-01,BEV,20601,21,Electric Vehicle Supply Equipment Tax Credit P...,Sheehy Auto Store,20601.0,Public,2020,7,20601.0,38.61863,-76.85715,Waldorf,MD,27340.0,220.7
1,2020-07-01,BEV,20601,21,Electric Vehicle Charging Stations,Waldorf Ford,20601.0,Public,2020,7,20601.0,38.61863,-76.85715,Waldorf,MD,27340.0,220.7
2,2020-07-01,BEV,20601,21,Electric Vehicle Infrastructure Program (EVIP),Charles County Hilton,20601.0,Public,2020,7,20601.0,38.61863,-76.85715,Waldorf,MD,27340.0,220.7
3,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Charles County Public Library - PD Brown Memor...,20602.0,Public,2020,7,20602.0,38.58335,-76.89466,Waldorf,MD,29425.0,784.8
4,2020-07-01,BEV,20602,26,Electric Vehicle Charging Stations,Walgreens,20602.0,Public,2020,7,20602.0,38.58335,-76.89466,Waldorf,MD,29425.0,784.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25234,2025-03-01,PHEV,21771,220,Electric Vehicle Charging Stations,Century Ford,21771.0,Public,2025,3,21771.0,39.39550,-77.15869,Mount Airy,MD,32827.0,141.6
25235,2025-03-01,PHEV,21784,233,Electric Vehicle Charging Stations,Walgreens,21784.0,Public,2025,3,21784.0,39.40087,-76.97260,Sykesville,MD,37813.0,260.1
25236,2025-03-01,PHEV,21801,126,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21801.0,Public,2025,3,21801.0,38.38106,-75.64073,Salisbury,MD,30063.0,289.4
25237,2025-03-01,PHEV,21804,76,Biodiesel Refueling Stations,Cato Pacific Pride Fuel,21804.0,Public,2025,3,21804.0,38.31726,-75.53673,Salisbury,MD,39461.0,222.1


In [51]:
path_1901 = './data/1901.csv'

def proc_df(path):
    df = pd.read_csv(path, skiprows=1)
    drop_col = []
    for col in df.columns:
        new_name = col.replace('!!',' ')
        if 'Margin of Error' in new_name or 'PERCENT ALLOCATED' in new_name:
            drop_col.append(new_name)
        df.rename(columns={col: new_name}, inplace=True)
    df.drop(columns=drop_col, inplace=True)
    df['Geographic Area Name'] = df['Geographic Area Name'].str.replace('ZCTA5 ','').astype(int)
    df.drop(columns = ['Geography'], inplace=True)
    df.dropna(axis=1, inplace=True)
    df = df.rename(columns={'Geographic Area Name': 'Zip_Code'})
    return df

In [52]:
df1901 = proc_df(path_1901)
df1901.head()

,Zip_Code,Estimate Households Total,"Estimate Households Total Less than $10,000","Estimate Households Total $10,000 to $14,999","Estimate Households Total $15,000 to $24,999","Estimate Households Total $25,000 to $34,999","Estimate Households Total $35,000 to $49,999","Estimate Households Total $50,000 to $74,999","Estimate Households Total $75,000 to $99,999","Estimate Households Total $100,000 to $149,999",...,"Estimate Nonfamily households Total $15,000 to $24,999","Estimate Nonfamily households Total $25,000 to $34,999","Estimate Nonfamily households Total $35,000 to $49,999","Estimate Nonfamily households Total $50,000 to $74,999","Estimate Nonfamily households Total $75,000 to $99,999","Estimate Nonfamily households Total $100,000 to $149,999","Estimate Nonfamily households Total $150,000 to $199,999","Estimate Nonfamily households Total $200,000 or more",Estimate Nonfamily households Median income (dollars),Estimate Nonfamily households Mean income (dollars)
0,19973,9927,6.1,4.6,7.0,8.6,15.0,17.0,11.4,15.3,...,11.9,16.5,18.8,12.2,9.6,2.7,3.9,2.6,34794,48087
1,20601,10011,3.1,2.4,2.6,2.3,4.7,9.3,15.5,29.1,...,7.5,3.9,8.9,10.6,21.9,24.1,8.5,5.0,83052,92628
2,20602,10659,5.3,3.1,2.9,2.8,6.1,12.6,10.9,27.6,...,5.8,5.6,10.1,18.9,10.5,24.2,5.0,1.4,67977,69240
3,20603,11518,2.0,1.8,3.2,3.0,4.0,8.3,11.0,21.8,...,6.8,6.7,7.2,12.6,13.0,21.1,15.8,6.4,81921,93785
4,20606,88,0.0,0.0,0.0,0.0,0.0,29.5,36.4,0.0,...,0.0,0.0,0.0,0.0,86.5,0.0,13.5,0.0,-,N


In [53]:
df1903 = proc_df('./data/1903.csv')
df1903.head()

,Zip_Code,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- White,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- Black or African American,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- American Indian and Alaska Native,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- Asian,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- Native Hawaiian and Other Pacific Islander,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households One race-- Some other race,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households Two or more races,Estimate Number HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER Households Hispanic or Latino origin (of any race),...,Estimate Median income (dollars) FAMILY INCOME BY NUMBER OF EARNERS 1 earner,Estimate Median income (dollars) FAMILY INCOME BY NUMBER OF EARNERS 2 earners,Estimate Median income (dollars) FAMILY INCOME BY NUMBER OF EARNERS 3 or more earners,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Female householder,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Female householder Living alone,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Female householder Not living alone,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Male householder,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Male householder Living alone,Estimate Median income (dollars) NONFAMILY HOUSEHOLDS Nonfamily households Male householder Not living alone
0,19973,9927,6823,2020,4,72,0,288,720,788,...,54238,101864,139750,34794,32411,31382,-,37233,36443,-
1,20601,10011,2825,5778,68,287,4,370,679,804,...,101250,144789,180184,83052,89229,85333,147059,77746,66875,94516
2,20602,10659,2587,6805,80,235,0,220,732,744,...,76797,147642,173438,67977,71263,61705,108533,64605,55455,112857
3,20603,11518,2565,7403,0,652,0,185,713,709,...,93719,186288,228466,81921,79010,72500,146591,84286,79393,213393
4,20606,88,81,7,0,0,0,0,0,0,...,-,-,-,-,-,-,-,-,-,-


In [54]:
df2503 = proc_df('./data/2503.csv')
df2503.head()

,Zip_Code,Estimate Occupied housing units Occupied housing units,"Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) Less than $5,000","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $5,000 to $9,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $10,000 to $14,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $15,000 to $19,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $20,000 to $24,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $25,000 to $34,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $35,000 to $49,999","Estimate Occupied housing units Occupied housing units HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS) $50,000 to $74,999",...,"Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $50,000 to $74,999","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $50,000 to $74,999 Less than 20 percent","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $50,000 to $74,999 20 to 29 percent","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $50,000 to $74,999 30 percent or more","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $75,000 or more","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $75,000 or more Less than 20 percent","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $75,000 or more 20 to 29 percent","Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS $75,000 or more 30 percent or more",Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS Zero or negative income,Estimate Percent renter-occupied housing units Occupied housing units MONTHLY HOUSING COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS No cash rent
0,19973,9927,419,189,456,362,333,853,1487,1688,...,17.0,0.9,8.8,7.3,19.6,15.1,2.3,2.2,0.9,4.9
1,20601,10011,223,87,245,72,191,231,473,932,...,4.3,0.0,0.0,4.3,69.3,21.1,39.1,9.0,1.8,4.9
2,20602,10659,302,258,330,176,130,303,651,1345,...,16.8,1.0,3.5,12.4,47.7,13.0,31.5,3.2,2.8,0.7
3,20603,11518,112,113,213,206,162,341,465,952,...,13.9,0.0,5.0,8.9,50.7,21.4,23.2,6.1,0.0,2.6
4,20606,88,0,0,0,0,0,0,0,26,...,-,-,-,-,-,-,-,-,-,-
